In [58]:
# Se impotan las librerias de manipulación de datos #
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble

In [59]:
# Importe de Dataframe "train" (entrenamiento) #
df_train = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRL55FMYP5fSEdsTj1aBG9uBIklH_sMqBlZdcNg0fgsPiND863gaiG_0uIiR9unsfAjGSEJqNiKuk19/pub?output=csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [60]:
# Se elimina las variables con baja correlación en dataframe TRAIN#
Variable_elimin_train=['Id','MSSubClass','OverallCond','BsmtFinSF2','LowQualFinSF','BsmtHalfBath','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']
train_x=df_train.drop(labels=Variable_elimin_train, axis=1)
train_x.columns

Index(['MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinType2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'PoolQC',
       'Fence', 'MiscFeature'

In [61]:
# Se eliminan filas con valores objetivos faltantes #
train_x.dropna(axis=0, subset=['SalePrice'], inplace=True)
# Se define la variable target #
vt = train_x.SalePrice
# Se elimina la variable target d elos datos de entrenamiento #
train_x.drop(['SalePrice'], axis=1, inplace=True)

In [62]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 68 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1460 non-null   object 
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   object 
 4   Alley          91 non-null     object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  YearBuilt      1460 non-null   int64  
 17  YearRemodAdd   1460 non-null   int64  
 18  RoofStyl

In [63]:
# Se elimna las columnas con valores nulos Nan #
train_x.drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'], axis=1, inplace=True)

In [64]:
# Se busca las columnas que tengan valores numericos #
col_num_train = train_x.select_dtypes(include=['number'])
col_num_train.columns

Index(['LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
       'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch'],
      dtype='object')

## **K-FOLD**

In [65]:
# Se realiza la cross validation #
# Se usara un folds de 5 a 10 #
kf =KFold(n_splits=10, shuffle=True, random_state=42)

# Se inicia con un contador = 1 #"
cnt = 1
# Se genera el metodo split() para dividir los datos en conjuntos #
for train_index, test_index in kf.split(col_num_train, vt):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 1314, Test set:146
Fold:2, Train set: 1314, Test set:146
Fold:3, Train set: 1314, Test set:146
Fold:4, Train set: 1314, Test set:146
Fold:5, Train set: 1314, Test set:146
Fold:6, Train set: 1314, Test set:146
Fold:7, Train set: 1314, Test set:146
Fold:8, Train set: 1314, Test set:146
Fold:9, Train set: 1314, Test set:146
Fold:10, Train set: 1314, Test set:146


In [66]:
# Se define la funcion error cuadratico medio #
def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

**REGRESION LINEAL**

In [67]:
# Se realiza la validadción cruzada k-fold de un modelo de regresión lineal #
# Se utiliza la función cross_val_score para validar #
score = cross_val_score(linear_model.LinearRegression(), col_num_train, vt, cv= kf, scoring="neg_mean_squared_error")
# Se imprime las puntuaciones para cada fold #
print(f'Puntuacion para cada fold: {score}')
# Se calcua el rmse promedio #
rmse(score.mean())

Puntuacion para cada fold: [-1.74037770e+09 -1.10104295e+09 -8.44597177e+08 -1.40484740e+09
 -1.88184284e+09 -3.89444860e+09 -1.30966618e+09 -7.05732050e+08
 -7.86035018e+08 -6.88836785e+08]
rmse= 37891.20


In [68]:
resultados = np.array([-1.74037770e+09, -1.10104295e+09, -8.44597177e+08, -1.40484740e+09,
                       -1.88184284e+09, -3.89444860e+09, -1.30966618e+09, -7.05732050e+08,
                       -7.86035018e+08, -6.88836785e+08])
for i, resultado in enumerate(resultados, start=1):
    print(f'Fold {i}: {resultado}')

Fold 1: -1740377700.0
Fold 2: -1101042950.0
Fold 3: -844597177.0
Fold 4: -1404847400.0
Fold 5: -1881842840.0
Fold 6: -3894448600.0
Fold 7: -1309666180.0
Fold 8: -705732050.0
Fold 9: -786035018.0
Fold 10: -688836785.0


* Cuanto mas cercano a cero se encuentre la puntuacion, mejor sera el rendimiento del modelo.

* La raíz cuadrada del error cuadrático medio (RMSE) promedio es 37891.20. Significa que, en promedio, las predicciones del modelo tienen un error de aproximadamente 37891.20 unidades en la misma escala que la variable objetivo.

* La variabilidad en las puntuaciones entre los folds puede indicar que el modelo se comporta de manera diferente en diferentes conjuntos de datos de entrenamiento y prueba.

**ARBOL DE DECISION**

In [76]:
# Se utiliza la función cross_val_score para validar #
score = cross_val_score(tree.DecisionTreeRegressor(random_state= 42), col_num_train, vt, cv=kf, scoring="neg_mean_squared_error")
df_scores = pd.DataFrame({'Puntuacion': score})
print(df_scores)
rmse(score.mean())

     Puntuacion
0 -2.298927e+09
1 -1.714501e+09
2 -1.966254e+09
3 -1.485443e+09
4 -1.769850e+09
5 -1.680118e+09
6 -1.515635e+09
7 -1.586336e+09
8 -1.241720e+09
9 -9.814848e+08
rmse= 40299.22


*  Los valores son bastante grandes en términos absolutos (en la escala de 10^9), pero como son negativos, es decir se esta tratando con una métrica de error cuadrático medio, por lo tanto se puede ignorar el signo negativo al interpretar la calidad del modelo.

* El valor de rmse=40299.22 indica la raíz cuadrada del error cuadrático medio promedio de todos los folds. Este valor proporciona una medida más interpretable del rendimiento del modelo.